In [0]:
bronze_path = "s3://capstone-impacta/Capstone/bronze"

df_bronze = (
    spark.read
         .json(bronze_path)
)



In [0]:
from pyspark.sql.functions import col, to_timestamp, to_date

df_silver = (
    df_bronze
    .withColumn("posted_at", to_timestamp("posted_at"))
    .withColumn("data_postagem", to_date("data_postagem"))
    .withColumn("ingestion_ts", to_timestamp("ingestion_ts"))
    .select(
        "faculdade",
        "rede_social",
        "data_postagem",
        "posted_at",
        "video_id",
        "titulo_video",
        "categoria",
        "duracao_seg",
        "views",
        "likes",
        "comentarios",
        "engagement_rate",
        "leads_estimados",
        "ctr_estimado",
        "run_id",
        "ingestion_ts"
    )
)


In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

w = Window.partitionBy("video_id").orderBy(col("ingestion_ts").desc())

df_silver_dedup = (
    df_silver
    .withColumn("rn", row_number().over(w))
    .filter(col("rn") == 1)
    .drop("rn")
)



In [0]:
silver_path = "s3://capstone-impacta/Capstone/silver/posts"

(
    df_silver_dedup
    .write
    .format("delta")
    .mode("overwrite")
    .partitionBy("rede_social", "data_postagem")
    .save(silver_path)
)


In [0]:
#silver_path = "s3://capstone-impacta/Capstone/silver/posts"
#spark.read.format("delta").load(silver_path).display()
